In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt.pyll.base import scope


import mag
from mag.experiment import Experiment
from mag import summarize


from speech2phone.preprocessing import get_TIMIT, get_phones
from speech2phone.preprocessing.filters import mel

np.set_printoptions(precision=3)

/Users/Seong-EunCho/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Random Forest with `mag`

In [2]:
X_toy, y_toy = get_TIMIT(dataset='toy', preprocessor=mel, TIMIT_root='../TIMIT/TIMIT')

Loading toy/mel set from cache... done.


In [3]:
def random_forest_acc(n_estimators, max_depth):
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_toy, y_toy)
    acc = rf.score(X_toy, y_toy)
    return acc



experiments_dir = '../experiments/results/random_forest_v1'
config = {
    'n_estimators': 11,
    'max_depth': 10
}
with Experiment(config=config, experiments_dir=experiments_dir) as experiment:
    config = experiment.config
    acc = random_forest_acc(config.n_estimators, config.max_depth)
    
    experiment.register_result('accuracy', acc)

ValueError: Experiment with identifier 10|11 already exists. Set `resume_from` to the corresponding identifier (directory name) 10|11 or delete it manually and then rerun the code.

# Random Forest with `hyperopt`

In [7]:
def acc_wrapper(args):
    acc = random_forest_acc(args['n_estimators'], args['max_depth'])
    print("{}: {:.3f}".format(args, acc))
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=50)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))

{'max_depth': 11, 'n_estimators': 73}: 0.905
{'max_depth': 15, 'n_estimators': 76}: 0.999
{'max_depth': 20, 'n_estimators': 75}: 1.000
{'max_depth': 18, 'n_estimators': 42}: 1.000
{'max_depth': 17, 'n_estimators': 68}: 1.000
{'max_depth': 8, 'n_estimators': 63}: 0.563
{'max_depth': 1, 'n_estimators': 20}: 0.135
{'max_depth': 26, 'n_estimators': 83}: 1.000
{'max_depth': 5, 'n_estimators': 14}: 0.308
{'max_depth': 14, 'n_estimators': 42}: 0.992
{'max_depth': 27, 'n_estimators': 33}: 1.000
{'max_depth': 4, 'n_estimators': 27}: 0.268
{'max_depth': 20, 'n_estimators': 6}: 0.957
{'max_depth': 12, 'n_estimators': 26}: 0.931
{'max_depth': 6, 'n_estimators': 31}: 0.382
{'max_depth': 11, 'n_estimators': 26}: 0.877
{'max_depth': 27, 'n_estimators': 88}: 1.000
{'max_depth': 23, 'n_estimators': 43}: 1.000
{'max_depth': 11, 'n_estimators': 54}: 0.897
{'max_depth': 3, 'n_estimators': 98}: 0.243
{'max_depth': 19, 'n_estimators': 56}: 1.000
{'max_depth': 29, 'n_estimators': 42}: 1.000
{'max_depth': 24,

# Random Forest with `mag` and `hyperopt`

In [6]:
def acc_wrapper(args):
    experiments_dir = '../experiments/results/random_forest_v2'
    with Experiment(config=args, experiments_dir=experiments_dir) as experiment:
        config = experiment.config
        acc = random_forest_acc(config.n_estimators, config.max_depth)
        experiment.register_result('accuracy', acc)
    return -acc

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators', 1, 100)),
    'max_depth': scope.int(hp.uniform('max_depth', 1, 30))

}
best = fmin(acc_wrapper, space, algo=tpe.suggest, max_evals=5)
print("")
print("Best Raw:", best)
print("Best Readable:", space_eval(space, best))


Best Raw: {'max_depth': 22.532586122234616, 'n_estimators': 41.60007385667237}
Best Readable: {'max_depth': 22, 'n_estimators': 41}
